# Exploring the possibilities

I want to build a database (SQL) of different simulations. 

## 2 qubit model
I consider a simple simulation involving two qubits. The Hamiltonian reads
$$
H = 
J \left(\sigma_z \otimes \sigma_z\right) + 
B \left(\sigma_x \otimes 1 + 1 \otimes \sigma_x\right)
$$
Additionally, there is local decoherence on each qubit given by jump operators $\sqrt{\Gamma} \left(\sigma_- \otimes 1\right)$ and $\sqrt{\Gamma} \left(1 \otimes \sigma_-\right)$. The initial state is $|\uparrow_z\rangle \otimes |\uparrow_z\rangle$.

## Collective spin model
I also consider a simple situation involving a collective spin with a linear term and a quadratic term. The Hamiltonian reads
$$
H = J S_z^2 + B S_x
$$
Additionally, there is collective decoherence given by jump operators $\sqrt{\Gamma} S_-$.

# Implementation questions
1. Consider a few different methods of saving simulation data to a table.
    1. Put the `qutip.solver.Result` object in the SQL database. I am storing everything from the simulation, I can store the time stamps for the starting and ending times.
        - Run a simulation.
        - Convert the `qutip.solver.Result` into `bytes` using `pickle.dumps`.
        - Save these `bytes` objects as `BLOB` values in a column of an SQL table.
    2. Put the `qutip.qobj.Qobj` objects representing the state in the SQL database.
        - Run a simulation. 
        - Extract the states as `qutip.qobj.Qobj`.
        - Convert the states into `bytes` using `pickle.dumps`. 
        - Save these `bytes` objects as `BLOB` values in a column of an `pandas.DataFrame`
        - Save the `pandas.DataFrame` to an SQL table
2. How close are the density matrices estimated by monte carlo wavefunction or quantum trajectories using `qutip.mcsolve` and solving the master equation using `qutip.mesolve`?

 

In [1]:
import qutip
import numpy as np
import itertools

import uuid
import time

import sqlite3
import pandas as pd
import pickle

from numpy import pi, sqrt

In [2]:
def save_result_blob (string_uuid, hamiltonians, ρ_initial, tlist, jumops, method, db_filename):
    
    # Run the simulation
    timestamp_start = time.time_ns()
    
    if method.lower() == 'me':
        result = qutip.mesolve(hamiltonians, ρ_initial, tlist, jumpops)
    elif method.lower() == 'mc':
        result = qutip.mcsolve(hamiltonians, ρ_initial, tlist, jumpops, ntraj=11)

    timestamp_end = time.time_ns()

    # Convert the `qutip.solver.Result` object to `bytes
    result_bytes = pickle.dumps(result)
    
    simulation_row = (string_uuid, timestamp_start, timestamp_end, result_bytes)
    
    # Save the row to a table of a SQL database
    if db_filename == None:
        return

    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()
    
    sql_create = '''
        CREATE TABLE IF NOT EXISTS SimulationSavedResults
        (uuid text,
        timestamp_start real,
        timestamp_end real,
        result blob)
    '''
    
    sql_insert = '''
        INSERT INTO SimulationSavedResults
        (uuid, timestamp_start, timestamp_end, result)
        VALUES
        (?, ?, ?, ?)'''
    
    cursor.execute(sql_create)
    cursor.execute(sql_insert, simulation_row)
    
    conn.commit()
    conn.close()

In [3]:
def save_state_blob (string_uuid, hamiltonians, ρ_initial, tlist, jumops, method, db_filename):
    
    # Run the simulation
    timestamp_start = time.time_ns()
    
    if method.lower() == 'me':
        result = qutip.mesolve(hamiltonians, ρ_initial, tlist, jumpops)
    elif method.lower() == 'mc':
        n_traj = 11
        result = qutip.mcsolve(hamiltonians, ρ_initial, tlist, jumpops, ntraj=n_traj)

    timestamp_end = time.time_ns()

    # Extract the states at different times
    n_steps = len(tlist)

    simulation_result_rows = []
    for s in range(n_steps):
        t = tlist[s]
        if method.lower() == 'me':
            # When solving the master equation, there is one state for each instance
            # of time. Therefore, there is one row for the state at each time.
            state = result.states[s]
            state_bytes = pickle.dumps(state)
            simulation_result_rows += [(string_uuid, t, None, state_bytes)]
        elif method.lower() == 'mc':
            # when using monte carlo wave function, there is one state for each instance
            # time and each trajectory. Therefore, there are multiple rows for the state
            # at each time
            for tr in range(n_traj):
                state = result.states[tr, s]
                state_bytes = pickle.dumps(state)
                simulation_result_rows += [(string_uuid, t, tr, state_bytes)]
                
                
    # Save the row to a table of a SQL database
    
    if db_filename == None:
        return
    
    simulations_df = pd.DataFrame(data=[(string_uuid, timestamp_start, timestamp_end)], columns=('uuid', 'timestamp_start', 'timestamp_end'))
    simulation_results_df = pd.DataFrame(data=simulation_result_rows, columns=('uuid', 'time', 'trajectory_index', 'state'))
    
    conn = sqlite3.connect(db_filename)
    
    simulations_df.to_sql('Simulations', con=conn, if_exists='append')
    simulation_results_df.to_sql('SimulationSavedStates', con=conn, if_exists='append')
    
    conn.commit()
    conn.close()
    
#     sql_create_simulations = '''
#         CREATE TABLE IF NOT EXISTS Simulations
#         (uuid text,
#         timestamp_start real,
#         timestamp_end real)
#     '''

#     sql_create_simulationresults = '''
#         CREATE TABLE IF NOT EXISTS SimulationSavedStates
#         (uuid text,
#         time real,
#         trajectory_index integer,
#         state blob)
#     '''
    
#     sql_insert_simulations = '''
#         INSERT INTO Simulations
#     '''

In [15]:
Ω = 1
κ = 1
Γ = 1e-2

t_initial = 0
t_final = 8 * 2*pi
n_steps = 128
n_traj = 11

t_list = np.linspace(t_initial, t_final, n_steps)

In [17]:
uuid_generated = uuid.uuid4() 
string_uuid = str(uuid_generated)

# This is the collective spin model
spin_j = 63/2
ket_initial = qutip.spin_coherent(spin_j, theta=0, phi=0)
hamiltonians = [Ω * qutip.spin_Jx(spin_j), κ * qutip.spin_Jz(spin_j) * qutip.spin_Jz(spin_j)]
jumpops = [sqrt(Γ) * qutip.spin_Jm(spin_j)]

In [18]:
db_filename = 'Benchmarking.db'

In [19]:
%%time
for _ in range(10):
    save_result_blob(string_uuid, hamiltonians, ket_initial, t_list, jumpops, 'mc', db_filename)

18.2%. Run time:   4.98s. Est. time left: 00:00:00:22
27.3%. Run time:   5.11s. Est. time left: 00:00:00:13
36.4%. Run time:   5.11s. Est. time left: 00:00:00:08
45.5%. Run time:   9.54s. Est. time left: 00:00:00:11
54.5%. Run time:   9.58s. Est. time left: 00:00:00:07
63.6%. Run time:   9.65s. Est. time left: 00:00:00:05
72.7%. Run time:   9.69s. Est. time left: 00:00:00:03
81.8%. Run time:  14.27s. Est. time left: 00:00:00:03
90.9%. Run time:  14.28s. Est. time left: 00:00:00:01
100.0%. Run time:  14.42s. Est. time left: 00:00:00:00
Total run time:  14.47s
18.2%. Run time:   4.59s. Est. time left: 00:00:00:20
27.3%. Run time:   4.68s. Est. time left: 00:00:00:12
36.4%. Run time:   4.74s. Est. time left: 00:00:00:08
45.5%. Run time:   9.42s. Est. time left: 00:00:00:11
54.5%. Run time:   9.48s. Est. time left: 00:00:00:07
63.6%. Run time:   9.54s. Est. time left: 00:00:00:05
72.7%. Run time:   9.71s. Est. time left: 00:00:00:03
81.8%. Run time:  13.97s. Est. time left: 00:00:00:03
90.

In [20]:
%%time
for _ in range(10):
    save_result_blob(string_uuid, hamiltonians, ket_initial, t_list, jumpops, 'mc', None)

18.2%. Run time:   4.92s. Est. time left: 00:00:00:22
27.3%. Run time:   4.95s. Est. time left: 00:00:00:13
36.4%. Run time:   5.04s. Est. time left: 00:00:00:08
45.5%. Run time:   9.78s. Est. time left: 00:00:00:11
54.5%. Run time:  10.07s. Est. time left: 00:00:00:08
63.6%. Run time:  10.12s. Est. time left: 00:00:00:05
72.7%. Run time:  10.33s. Est. time left: 00:00:00:03
81.8%. Run time:  14.61s. Est. time left: 00:00:00:03
90.9%. Run time:  14.64s. Est. time left: 00:00:00:01
100.0%. Run time:  14.81s. Est. time left: 00:00:00:00
Total run time:  14.88s
18.2%. Run time:   4.88s. Est. time left: 00:00:00:21
27.3%. Run time:   4.92s. Est. time left: 00:00:00:13
36.4%. Run time:   5.10s. Est. time left: 00:00:00:08
45.5%. Run time:   9.66s. Est. time left: 00:00:00:11
54.5%. Run time:   9.79s. Est. time left: 00:00:00:08
63.6%. Run time:   9.91s. Est. time left: 00:00:00:05
72.7%. Run time:  10.45s. Est. time left: 00:00:00:03
81.8%. Run time:  14.38s. Est. time left: 00:00:00:03
90.

In [21]:
%%time
for _ in range(10):
    save_state_blob(string_uuid, hamiltonians, ket_initial, t_list, jumpops, 'mc', db_filename)

18.2%. Run time:   5.11s. Est. time left: 00:00:00:22
27.3%. Run time:   5.17s. Est. time left: 00:00:00:13
36.4%. Run time:   5.28s. Est. time left: 00:00:00:09
45.5%. Run time:   9.82s. Est. time left: 00:00:00:11
54.5%. Run time:  10.12s. Est. time left: 00:00:00:08
63.6%. Run time:  10.29s. Est. time left: 00:00:00:05
72.7%. Run time:  10.33s. Est. time left: 00:00:00:03
81.8%. Run time:  14.69s. Est. time left: 00:00:00:03
90.9%. Run time:  14.79s. Est. time left: 00:00:00:01
100.0%. Run time:  15.19s. Est. time left: 00:00:00:00
Total run time:  15.28s
18.2%. Run time:   4.92s. Est. time left: 00:00:00:22
27.3%. Run time:   5.00s. Est. time left: 00:00:00:13
36.4%. Run time:   5.09s. Est. time left: 00:00:00:08
45.5%. Run time:   9.48s. Est. time left: 00:00:00:11
54.5%. Run time:   9.76s. Est. time left: 00:00:00:08
63.6%. Run time:  10.04s. Est. time left: 00:00:00:05
72.7%. Run time:  10.34s. Est. time left: 00:00:00:03
81.8%. Run time:  14.28s. Est. time left: 00:00:00:03
90.

In [22]:
%%time
for _ in range(10):
    save_state_blob(string_uuid, hamiltonians, ket_initial, t_list, jumpops, 'mc', None)

18.2%. Run time:   5.03s. Est. time left: 00:00:00:22
27.3%. Run time:   5.12s. Est. time left: 00:00:00:13
36.4%. Run time:   5.13s. Est. time left: 00:00:00:08
45.5%. Run time:   9.81s. Est. time left: 00:00:00:11
54.5%. Run time:  10.01s. Est. time left: 00:00:00:08
63.6%. Run time:  10.12s. Est. time left: 00:00:00:05
72.7%. Run time:  10.40s. Est. time left: 00:00:00:03
81.8%. Run time:  14.55s. Est. time left: 00:00:00:03
90.9%. Run time:  14.77s. Est. time left: 00:00:00:01
100.0%. Run time:  15.08s. Est. time left: 00:00:00:00
Total run time:  15.11s
18.2%. Run time:   5.00s. Est. time left: 00:00:00:22
27.3%. Run time:   5.09s. Est. time left: 00:00:00:13
36.4%. Run time:   5.13s. Est. time left: 00:00:00:08
45.5%. Run time:   9.96s. Est. time left: 00:00:00:11
54.5%. Run time:  10.05s. Est. time left: 00:00:00:08
63.6%. Run time:  10.19s. Est. time left: 00:00:00:05
72.7%. Run time:  10.48s. Est. time left: 00:00:00:03
81.8%. Run time:  14.87s. Est. time left: 00:00:00:03
90.

In [11]:
ρ_trajectories_mc = np.empty((n_traj, n_steps), dtype=object)

for tr, st in itertools.product(range(n_traj), range(n_steps)):
     #ρ_estimates_mc[tr, st] = qutip.ket2dm(results_mc.states[tr, st])
    pass